In [ ]:
%run Utils/Utils.ipynb

In [ ]:
''' If you ever have trouble with lidar, try running the code below. It should work after 1 or 2 tries.
If you get error about no device being connected, try using port 0, 1 or 2.'''
# from rplidar import RPLidar
# lidar = RPLidar('/dev/ttyUSB1')

# info = lidar.get_info()
# print(info)

# health = lidar.get_health()
# print(health)

# for i, scan in enumerate(lidar.iter_scans()):
#     print('%d: Got %d measurments' % (i, len(scan)))
#     if i > 1000:
#         break

# lidar.stop()
# lidar.stop_motor()
# lidar.disconnect()

In [ ]:
%run Utils/Utils_Widgets.ipynb

In [ ]:
# %run Utils/Utils_DepthCam.ipynb
%run Utils/Utils_Camera.ipynb

In [ ]:
#!pip install paho.mqtt
%run Utils/Utils_MQTT.ipynb
stepCount = 1

# Parameters

In [ ]:
import copy

goal_x_coor = 0 #-1.95 #1.6  # in m, measured relative to the initial robot coordinates
goal_y_coor = 0.80 #5.5 #5.20  # in m, measured relative to the initial robot coordinates

# Save the original goal location. This will help us to continuously go back to the 
# original X and Y coordinates we set above. 
orig_x_coor = copy.deepcopy(goal_x_coor) 
orig_y_coor = copy.deepcopy(goal_y_coor) 

running_control_thread = False

getObservation()
print(observation)

In [ ]:
# model_path = "Turtlebot3_0331.pth"
model_path = "Turtlebot3_LSTM.pth"
agent = torch.load(model_path)
agent.eval()
print('Model Loaded!')

In [ ]:
#getObservation()
memories = torch.ones((1,1,256))
masks = torch.ones((1,6))

# Initialize Logging

In [ ]:
import logging
import datetime

now = datetime.datetime.now()
dt_string = now.strftime("%Y%m%d__%H_%M_%S")

In [ ]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(filename="log_{}.txt".format(dt_string), level=logging.ERROR, format="%(asctime)s %(message)s")
logging.error("{},{},{},{}".format(real_x, real_y, real_angle, stepCount))

In [ ]:
# Test

# observation = [1]*41

# dists, value_out, memories = agent.get_dist_and_value(vec_inputs=[torch.Tensor(observation).cuda()], vis_inputs=[], masks=masks, memories=memories)
# actions = GetActionDiscrete(agent)  # LSTM
# # actions = GetAction(agent)          # MLP
# actions

# for dist in dists:
#     print(dist.entropy())
    
# for dist in dists:
#     print(dist.probs)
#     dist.probs=dist.probs*5
    
#     print(dist.probs)
    
# for dist in dists:
#     print(dist.probs)
    
# for dist in dists:
#     print(dist.entropy())
    
# actions = []
# for dist in dists:
#     actions.append(dist.sample().cpu().numpy()[0][0])
# print(actions)

# Control

In [ ]:
def ControlRobot(debug=False):
    
    global stepCount
    global goal_x_coor, goal_y_coor
    global orig_x_coor, orig_y_coor

    delay_counter = 0
    
    
    
    # When lap number is even, it means we are going towards the original goal
    # when it is odd, it means we are going back towards the origin
    lapNumber = 2 

    while True:
        
        # If we are in an even lap number, make the goal the original goal location
        if lapNumber % 2 == 0:
            goal_x_coor = copy.deepcopy(orig_x_coor)
            goal_y_coor = copy.deepcopy(orig_y_coor)
            
        # Otherwise, make the goal the origin location
        else:
            goal_x_coor = 0
            goal_y_coor = 0
            
        # Update the state vector before checking the angle!
        getObservation()
            
        # Rotate the robot!!!
        while observation[3] > 20 or observation[3] < -20:
            
            print("Rotating. Current angle to new goal = ", observation[3])

            if observation[3] > 0:  # Right
                Rotate(degrees=10, direction='right', speed=0.20)

            elif observation[3] < 0:  # Left
                Rotate(degrees=10, direction='left', speed=0.20)
                
            # Update the state vector after rotating!
            getObservation()
            
            
        # When the robot reaches goal, this becomes false and robot rotates. 
        moving = True  
        
        print(f"Goal x = {goal_x_coor}, goal y = {goal_y_coor}, angle to goal = {observation[3]}")

        # As long as we are inside this loop, our robot will move towards the goal
        while moving == True:

            # Get an Observation
            try:
                getObservation()
            except:
                continue

            updateStateWidgets()

            # Get Action
            #actions = GetAction(agent)         # Get act ion from MLP
            #actions = GetActionDiscrete(agent)  # Get action from LSTM

            actions = GetActionMod(agent)      # Get action from LSTM

            # Add a short delay at the beginning to give enough time for everything to load
            if delay_counter>5:

                # If still far away
                if observation[0] > 0.40:
                    #print(f'Performing action = {actions}')
                    act(actions)
                    #a=0

                # If you enter here, it means you have reached the goal
                else:
                    print("reached goal! Time to rotate.")
                    
                    # Increase the lap number 
                    lapNumber += 1 
                    
                    # Stop moving towards the goal, and exit this while to start rotating
                    # towards the opposite direction! :) 
                    moving = False

            delay_counter+=1
            

In [ ]:
def showDepthcamFeed_TRT_CSI():
    global clockIsVisible, image
    global running_control_thread
    
    while True:

        # Obtain BBoxes + Labels from model
        detections = getDetections(image)
        
        # 85 = clock, 
        if 85 in detections : # or 31 in detections or 25 in detections or 85 in detections:
            clockIsVisible =1 
        else:
            clockIsVisible = 0
        
            
        # Run Control Thread
        if running_control_thread == False:
            
            controlThread = Thread(target = ControlRobot)
            controlThread.start()
            running_control_thread = True

In [ ]:
stepCount

In [ ]:
display(full_widget)

In [ ]:
# Delay Camera thread to make sure it works.
display(image_widget)

In [ ]:
# Start camera
execute({'new': camera.value})
camera.observe(execute, names='value')

In [ ]:
# showCSIFeed()
showDepthcamFeed_TRT_CSI()